# CCTag Handeye Calibration
This notebook shows how to perform hand-eye calibration with CCTag pattern.

Hand-eye calibration is an important task in computer vision and robotics, which aims to determine the spatial relationship between the camera and the robot's end effector. This process is crucial for achieving accurate grasping, positioning, and manipulation.

The CCTag pattern consists of a set of concentric circles with different radii, forming a unique pattern. This design allows the tag to be recognized at different scales and viewing angles while maintaining high recognition accuracy. Each CCTag pattern has a central circle surrounded by several concentric rings, which encode information through the width, number and arrangement order of the rings.

## 1. Dependency import and description

In [1]:
import os
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import open3d.core as o3c

import rlia

The `os` module is a part of the Python standard library that provides a way to interact with the operating system. It offers functions for navigating the file system, manipulating file paths, and managing environment variables.

The `json` module is also a part of the Python standard library and is used for working with JSON (JavaScript Object Notation) data. It provides methods for serializing Python objects into JSON format and deserializing JSON data back into Python objects.

The `pickle` module is a Python standard library that allows for the serialization and deserialization of Python objects. It is commonly used to save and load complex data structures, such as the calibration data we will be working with in this notebook.

The `numpy` library is fundamental for numerical computations in Python. It provides support for arrays, matrices, and a collection of mathematical functions to operate on these data structures.

The `matplotlib.pyplot` module is a collection of functions that make matplotlib work like MATLAB. It is used for creating static, animated, and interactive visualizations in Python.

The `open3d` library is designed for processing 3D data. It supports 3D data visualization, point cloud processing, and mesh processing, making it essential for working with RGB-D data in robotic applications.

The `open3d.core` module provides lower-level core functionality for Open3D, including tensor operations and low-level data structures. It is used for efficient processing of large-scale 3D data.

The `rlia` library is a Robot Library for Indoor Applications. It includes various modules for data handling, calibration, and utility functions, which are vital for implementing the hand-eye calibration process.

# 2. Helper Functions
Code for visualization, transform rgbd data to point cloud.

In [ ]:
def depth_rgb_to_pc(depth_map:np.array, rgb:np.array, intrinsic:np.array) -> o3d.geometry.PointCloud:
    """ Convert RGB-D images to a point cloud.  
  
    Args:  
        depth_map (np.array): A 2D array representing depth values (Z) of the scene.  
        rgb (np.array): A 2D or 3D array representing RGB color values of the scene.   
                    If 2D, it is assumed to be a grayscale image that will be repeated to create an RGB image.  
        intrinsic (np.array): A 3x3 intrinsic camera matrix.  
  
    Returns:  
        o3d.geometry.PointCloud: A point cloud object containing 3D points and their corresponding RGB colors.  
    """  
    # Get the resolution of the depth map  
    resolution_x, resolution_y = depth_map.shape  
      
    # Flatten the depth map to a 1D array  
    real_depth = depth_map.reshape(resolution_x * resolution_y)  
  
    # Check if the RGB image is in color or grayscale  
    if (len(rgb.shape) == 3):  # Color image  
        rgb_list = rgb.reshape(resolution_x * resolution_y, 3) / 255.0  # Normalize RGB values  
    elif (len(rgb.shape) == 2):  # Grayscale image  
        rgb_copy = np.repeat(rgb[:, :, None], 3, axis=2)  # Repeat grayscale values to create an RGB image  
        rgb_list = rgb_copy.reshape(resolution_x * resolution_y, 3) / 255.0  # Normalize RGB values  
  
    # Create pixel indices for x and y coordinates  
    pixel_index = np.arange(resolution_x * resolution_y)  
    pixel_index_x = pixel_index % resolution_y  # X coordinate  
    pixel_index_y = pixel_index // resolution_y  # Y coordinate  
  
    def get_inv_intrinsic(intrinsic:np.array) -> np.array:
        """ Compute the inverse of the intrinsic camera matrix.

        Args:
            intrinsic (np.array): A 3x3 intrinsic camera matrix.  
    
        Returns:  
            np.array: The inverse of the intrinsic camera matrix, which is also a 3x3 matrix.  
        """  
        return np.array([[1 / intrinsic[0, 0], 0, -intrinsic[0, 2] / intrinsic[0, 0]],  # Inverse focal length and principal point x  
                        [0, 1 / intrinsic[1, 1], -intrinsic[1, 2] / intrinsic[1, 1]],  # Inverse focal length and principal point y  
                        [0, 0, 1]])  # Homogeneous coordinate for 2D to 3D transformation 
  
    # Compute the inverse of the intrinsic matrix  
    inv_intrinsic = get_inv_intrinsic(intrinsic)  
  
    # Create a homogeneous pixel coordinate matrix  
    pixel_homo_t = np.stack([pixel_index_x, pixel_index_y, np.ones(resolution_x * resolution_y)])  
  
    # Convert pixel coordinates to camera coordinates  
    camera_homo_t = inv_intrinsic @ pixel_homo_t  
  
    # Scale the camera coordinates by the depth values to obtain 3D points  
    camera_pc_t = camera_homo_t * real_depth  
  
    # Prepare the point cloud data in homogeneous coordinates  
    camera_pc_h_t = np.ones(shape=(camera_pc_t.shape[0] + 1, camera_pc_t.shape[1]), dtype=np.float32)  
    camera_pc_h_t[:3, :] = camera_pc_t  # Set the first three rows to the camera points  
  
    # Create an Open3D PointCloud object  
    camera_pc_o3d = o3d.geometry.PointCloud()  
    camera_pc_o3d.points = o3d.utility.Vector3dVector(camera_pc_t.T)  # Set the points  
    camera_pc_o3d.colors = o3d.utility.Vector3dVector(rgb_list)  # Set the colors  
  
    return camera_pc_o3d  # Return the point cloud object


## 3. Pattern Detect  
  
In this calibration process, we collect data for each hand-eye sample, which includes the board pose and the robot arm's end-effector pose. The DetectBoard function is utilized to detect the board pose from an RGB-depth image pair. 

This data is crucial for accurately establishing the geometric relationship between the camera and the robotic manipulator. 

- Pattern recognition includes two methods:

  - `Concentric circle detection for Monocular Camera` uses a single camera to identify and analyze concentric circle patterns in images. This method is commonly employed in calibration tasks for robotic systems and computer vision applications. 

  - `Concentric circle detection for Binocular Camera` employs two cameras to capture images of the same concentric circle patterns from different viewpoints. This approach allows for depth perception and improved accuracy in determining the 3D positions of detected features.

### 3.1 Concentric Circle Detection for Monocular Camera


#### Function: `rlia.calibration.ellipse_detect`
This function detects concentric circles (ellipses) in the provided RGB images from a monocular camera setup. It utilizes the intrinsic and distortion parameters of both cameras to accurately identify the circle patterns.  
  

##### Parameters  
- `rgb_t`: A list of RGB images provided as either [rows, cols] or [rows, cols, 3] uint8 format. This image is used to detect the concentric circles.
- `depthO`: A [rows, cols] float tensor representing the depth map associated with the RGB image. This depth information aids in accurately reconstructing the 3D positions of the detected circles.
- `k_t`: A [3, 3] double tensor representing the intrinsic camera matrix. This matrix includes parameters such as focal lengths and optical centers, which are essential for understanding how the camera captures the scene.
- `d_t`: A [5] double tensor representing the distortion coefficients. These coefficients account for lens distortion, including radial and tangential distortions, and are crucial for correcting the captured image.

##### Return
- `report`: Structure returned by `ellipse_detect`.  
  - `report.success`: Indicates the detect result status. 
  - `board_pose`: The pose of the center of the recognized concentric circles. The default position is np.eye(3)

In [ ]:
# Import necessary libraries  
import pickle  
import numpy as np  
import open3d as o3d
import open3d.core as o3c
import matplotlib.pyplot as plt  
import rlia  # Ensure you have the necessary rlia library  
  
# Load example data for hand-eye calibration  
example_datas = rlia.data.HandeyeCalibrate()  # Instantiate the calibration data handler  
example_path = example_datas.get_paths("single_marker")[0]  # Get the path for the single marker data  
  
# Load the calibration data from the pickle file  
with open(example_path, 'rb') as f:  
    example_data = pickle.load(f)  
  
# Extract intrinsic camera parameters, distortion coefficients, RGB, and depth images  
k = example_data["intrinsic"]  # Intrinsic camera matrix  
d = example_data["distortion"]  # Distortion coefficients  
rgb = example_data["rgb"][0]  # First RGB image  
depth = example_data["depth"][0]  # First depth image  
  
# Convert parameters and images to Open3D tensors  
k_t = o3c.Tensor(k, dtype=o3c.Dtype.Float64)  # Intrinsic matrix as tensor  
d_t = o3c.Tensor(d, dtype=o3c.Dtype.Float64)  # Distortion coefficients as tensor  
rgb_t = o3c.Tensor(rgb, dtype=o3c.Dtype.UInt8)  # RGB image as tensor  
depth_t = o3c.Tensor(depth, dtype=o3c.Dtype.Float32)  # Depth image as tensor  
  
# Detect the concentric circles in the RGB and depth images  
report = rlia.calibration.ellipse_detect(rgb_t, depth_t, k_t, d_t)  # Call the detection function  
  
# Check if the detection was successful  
if report.success:  
    print("Detected concentric circles")  # Indicate successful detection  
  
    # Prepare the RGB image for visualization  
    rgb_draw = rgb.copy()  # Create a copy of the RGB image for drawing  
    if rgb_draw.ndim == 2:  # If the image is grayscale  
        rgb_draw = np.repeat(rgb_draw[:, :, None], 3, axis=2)  # Convert to 3-channel RGB  
  
    # Convert the RGB image to tensor for drawing  
    rgb_draw_t = o3c.Tensor(rgb_draw, dtype=o3c.Dtype.UInt8)  
  
    # Draw the detected pose on the image  
    rgb_draw_t = rlia.calibration.DrawPoseOnImg(  
        rgb_t=rgb_draw_t,  
        k_t=k_t,  
        pose_t=report.board_pose,  
        axis_len=0.1  # Length of the axis to draw  
    )  
  
    # Display the image with the drawn pose  
    plt.imshow(rgb_draw_t.numpy())  
    plt.axis('off')  # Turn off axis labels  
    plt.show()  # Show the image  
  
    # Convert depth and RGB images to a point cloud  
    pc_o3d = depth_rgb_to_pc(depth, rgb_draw, k)  
    rlia.utility.draw([pc_o3d], backend='k3d', point_size=0.002) 
  
    # Output the calibrated position of the concentric circles  
    point_xyz = report.board_pose.numpy()  # Convert the pose tensor to a NumPy array  
    print("Calibration result, position of concentric circles:")  
    print(point_xyz)  # Print the detected positions  
else:  
    print("Failed to detect concentric circles")  # Indicate failure to detect 

### 3.2 Concentric Circle Detection for Binocular Camera


#### Function: `rlia.calibration.stereo_ellipse_detect`
This function detects concentric circles (ellipses) in the provided RGB images from a binocular camera setup. It utilizes the intrinsic and distortion parameters of both cameras to accurately identify the circle patterns.  
  
##### Parameters  
- `rgb_list`: A list of RGB images provided as either [rows, cols] or [rows, cols, 3] uint8 format. Each image should correspond to the left and right cameras in the binocular setup.  
- `k_list`: A list of [3, 3] double camera intrinsic matrices for each camera. These matrices define the internal parameters of the cameras, including focal lengths and optical centers.  
- `d_list`: A list of distortion coefficients for each camera, represented as [5] double matrices. The coefficients typically include:  
  - k1, k2: Radial distortion coefficients.  
  - p1, p2: Tangential distortion coefficients.  
  - k3: Additional radial distortion coefficient.  
- `R`: A [3, 3] double rotation matrix that describes the orientation of one camera relative to the other.  
- `T`: A [3] double translation matrix that specifies the position of one camera relative to the other (unit: meters).

##### Return
- `report`: Structure returned by `stereo_ellipse_detect`.  
  - `report.success`: Indicates the detect result status. 
  - `board_pose`: The pose of the center of the recognized concentric circles. The default position is np.eye(3)

In [ ]:
# Import necessary libraries  
import os  
import json  
import cv2  
import numpy as np  
import open3d as o3d  
import open3d.core as o3c  
import matplotlib.pyplot as plt  
import rlia  # Ensure you have the necessary rlia library  
np.set_printoptions(6, suppress=True)

# Load example data for hand-eye calibration  
example_datas = rlia.data.HandeyeCalibrate()  # Instantiate the calibration data handler  
data_dir = example_datas.extract_dir  # Extract the data directory path  
example_path = os.path.join(data_dir, "stereo_params.json")  # Define the path to the stereo parameters JSON file  
  
# Load the calibration data from the JSON file  
with open(example_path, 'r') as f:  
    example_data = json.load(f)  # Load the JSON content into a dictionary  
  
# Extract rotation matrix, translation vector, intrinsic parameters, and distortion coefficients  
r = example_data["R_l_r"]  # Rotation matrix from left to right camera  
t = example_data["t_l_r"]  # Translation vector from left to right camera  
k_left = example_data["cam1_k"]  # Intrinsic parameters of the left camera  
k_right = example_data["cam2_k"]  # Intrinsic parameters of the right camera  
d_left = example_data["dist_1"]  # Distortion coefficients for the left camera  
d_right = example_data["dist_2"]  # Distortion coefficients for the right camera  
  
# Load the stereo images from the data directory  
img_left = cv2.imread(os.path.join(data_dir, "stereo_left.bmp"))  # Read the left stereo image  
img_right = cv2.imread(os.path.join(data_dir, "stereo_right.bmp"))  # Read the right stereo image  
  
# Convert parameters and images to Open3D tensors  
r_t = o3c.Tensor(r, dtype=o3c.Dtype.Float64)  # Rotation matrix as tensor  
t_t = o3c.Tensor(t, dtype=o3c.Dtype.Float64).reshape(-1)  # Translation vector as tensor  
k_left_t = o3c.Tensor(k_left, dtype=o3c.Dtype.Float64)  # Left camera intrinsic parameters as tensor  
k_right_t = o3c.Tensor(k_right, dtype=o3c.Dtype.Float64)  # Right camera intrinsic parameters as tensor  
d_left_t = o3c.Tensor(d_left, dtype=o3c.Dtype.Float64).reshape(-1)  # Left camera distortion coefficients as tensor  
d_right_t = o3c.Tensor(d_right, dtype=o3c.Dtype.Float64).reshape(-1)  # Right camera distortion coefficients as tensor  
img_left_t = o3c.Tensor(img_left, dtype=o3c.Dtype.UInt8)  # Left image as tensor  
img_right_t = o3c.Tensor(img_right, dtype=o3c.Dtype.UInt8)  # Right image as tensor  
  
# Detect the concentric circles in the RGB images from both cameras  
report = rlia.calibration.stereo_ellipse_detect(  
    rgb_list=[img_left_t, img_right_t],  # List of RGB images  
    k_list=[k_left_t, k_right_t],  # List of intrinsic parameters  
    d_list=[d_left_t, d_right_t],  # List of distortion coefficients  
    R=r_t,  # Rotation matrix  
    T=t_t / 1000  # Translation vector, converted to meters (assuming input is in mm)  
)  # Call the detection function  
  
# Check if the detection was successful  
if report.success:  
    print("Detected concentric circles")  # Indicate successful detection  
  
    # Prepare the RGB image for visualization  
    rgb_draw = img_left.copy()  # Create a copy of the left RGB image for drawing  
    if rgb_draw.ndim == 2:  # If the image is grayscale  
        rgb_draw = np.repeat(rgb_draw[:, :, None], 3, axis=2)  # Convert to 3-channel RGB  
  
    # Convert the RGB image to tensor for drawing  
    rgb_draw_t = o3c.Tensor(rgb_draw, dtype=o3c.Dtype.UInt8)
  
    # Draw the detected pose on the image  
    rgb_draw_t = rlia.calibration.DrawPoseOnImg(  
        rgb_t=rgb_draw_t,  # Image tensor  
        k_t=k_left_t,  # Left camera intrinsic parameters tensor  
        pose_t=report.board_pose,  # Detected board pose  
        axis_len=0.1  # Length of the axis to draw (in meters)  
    )  
  
    # Display the image with the drawn pose  
    plt.imshow(rgb_draw_t.numpy())  # Convert the tensor to NumPy array for display  
    plt.axis('off')  # Turn off axis labels  
    plt.show()  # Show the image  
  
    # Output the calibrated position of the concentric circles  
    point_xyz = report.board_pose.numpy()  # Convert the pose tensor to a NumPy array  
    print("Calibration result, position of concentric circles:")  
    print(point_xyz)  # Print the detected positions  
else:  
    print("Failed to detect concentric circles")  # Indicate failure to detect

## Handeye Calibrate

In hand-eye calibration, the goal is to determine the relationship between a camera and a robotic manipulator. This is typically achieved using the AX = YB problem formulation, where:  
- A represents the poses of the calibration board relative to the camera.  
- B represents the poses of the robot's end effector relative to the robot base or a world coordinate system.  
  
##### 1. Concentric Circle Calibration for Monocular Camera
- For monocular setups, the following functions are primarily used:  
  - `rlia.calibration.ellipse_detect`: Detects concentric circles in the RGB and depth images.  
  - `rlia.calibration.CalibrateAxyb`: Solves the AX = YB problem to calibrate the camera and end effector poses.  
  
##### 2. Concentric Circle Calibration for Binocular Camera
- For binocular setups, the following functions are primarily employed:  
  - `rlia.calibration.stereo_ellipse_detect`: Detects concentric circles in the RGB images from both cameras.  
  - `rlia.calibration.CalibrateAxyb`: Solves the AX = YB problem similarly to the monocular approach.  

### Function: `rlia.calibration.CalibrateAxyb`

#### Parameters  
- `pose_board_t`: List of [4, 4] o3c.Tensor. Representing the poses of the calibration board relative to the camera coordinate system. Each pose includes both position and orientation. 
- `pose_ee_t`: List of [4, 4] o3c.Tensor. Representing the poses of the robot's end effector relative to the robot base or a world coordinate system calibration board and a robot's end effector.
- `is_eye_in_hand`: bool. 
  - If `True`, indicates the camera is mounted on the end effector (eye-in-hand configuration). 
  - If `False`, indicates the camera is fixed to the robot base (eye-to-hand configuration).

#### Return
- `report`: Structure returned by `CalibrateAxyb`.  
  - `report.status`: Indicates the calibration result status. A value of 0 indicates accurate calibration; any other value indicates inaccurate results.  
  - `report.pose_calibrate.numpy()`: The calibrated pose of the camera relative to the end effector or base, returned as a NumPy array.  
  - `report.reproject_err`: The translation error when the hand-eye transformation is reprojected to the end effector. 


## 4. Example

4.1 (`Monocular Camera`) Concentric Circle Handeye-Calibration Example 

In [ ]:
# Import necessary libraries  
import os  
import json  
import cv2  
import numpy as np  
import open3d as o3d  
import open3d.core as o3c  
import matplotlib.pyplot as plt  
import rlia  # Ensure you have the necessary rlia library 
np.set_printoptions(6, suppress=True)

example_datas = rlia.data.HandeyeCalibrate()  # hand eye Calibrate
example_path = example_datas.get_paths("single_marker")[0]

with open(example_path, 'rb') as f:  
    example_data = pickle.load(f) 

# Extract intrinsic and distortion parameters from the loaded data  
intrinsic = o3c.Tensor(example_data["intrinsic"], dtype=o3c.Dtype.Float64)  
distortion = o3c.Tensor(example_data["distortion"], dtype=o3c.Dtype.Float64)  
is_eye_in_hand = example_data["isEyeInHand"]  # Flag indicating if the camera is mounted on the end-effector

# Initialize lists to store poses and transformations  
board_pose_list = []  
ee_pose_list = []  
t_target_to_camera = []  
t_robot_pose = []  
t_target_cloud = [] 

# Get the number of RGB images available in the example data  
num_samples = len(example_data["rgb"]) 

# Iterate over each sample  
for i in range(num_samples):  
    # Convert RGB and depth images to tensors  
    rgb = o3c.Tensor(example_data["rgb"][i], dtype=o3c.Dtype.UInt8)  # RGB image tensor  
    depth = o3c.Tensor(example_data["depth"][i], dtype=o3c.Dtype.Float32)  # Depth image tensor  
  
    # Detect the board pose from the RGB and depth tensors  
    detect_report = rlia.calibration.ellipse_detect(rgb, depth, intrinsic, distortion)  
  
    # Store the detected board pose  
    board_pose_list.append(detect_report.board_pose)  
  
    # Convert the end-effector pose to a tensor and store it  
    ee_pose_tensor = o3c.Tensor(example_data["eePose"][i], dtype=o3c.Dtype.Float64)  
    ee_pose_list.append(ee_pose_tensor)  
  
    # Store the robot pose and the target cloud position  
    t_robot_pose.append(example_data["eePose"][i])  # Store the robot pose  
    t_target_cloud.append(detect_report.board_pose.numpy()[:, 3])  # Store the target cloud position  
    t_target_to_camera.append(detect_report.board_pose.numpy())  # Store the transformation from target to camera  
  
# Perform calibration using the collected board and end-effector poses  
calibrate_report = rlia.calibration.CalibrateAxyb(board_pose_list, ee_pose_list, is_eye_in_hand)  
  
# Print calibration results in a structured format  
print("Calibration Results:")  
print("\tCalibrated success:\t", calibrate_report.status == 0) 
print("\teprojection Error:\t", calibrate_report.reproject_err)
print("\tCalibrated Pose:\n", calibrate_report.pose_calibrate.numpy()) 

4.1 (`Binocular Camera`) Concentric Circle Handeye-Calibration Example 

- You can refer to (`Monocular Camera`) Concentric Circle Handeye-Calibration Example, and then change the recognition function from `rlia.calibration.ellipse_detect` to `rlia.calibration.stereo_ellipse_detect`.